## Importing Dependencies

In [2]:
import requests
from htmldate import find_date
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup
import ssl
from tqdm import tqdm
from textblob import TextBlob

from html_extractor import *
from get_suburls import *

from keyword_extraction import keyword_extractor_paragraph as kep

## Extracting sub urls

In [3]:
urls_list = ["https://www.khaleejtimes.com"]
scraper = WebScraper(1) # KEEP IT 1, 2 or more will result in 1000's of urls.
                        # Integration with DB will make it faster in future, as fetching is much faster than scrapping.
inside_urls, failed_fetch, sub_url_size, total_size = scraper.get_suburls(urls_list)

print("Inside URLs:", inside_urls)
print("Failed Fetch:", failed_fetch)
print("Splits:", len(inside_urls))
print("Tree size:", total_size)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.11it/s]

Inside URLs: {0: ['https://www.khaleejtimes.com'], 1: ['https://www.khaleejtimes.com/mykt', 'https://www.khaleejtimes.com/trading', 'https://www.khaleejtimes.com/contact-us', 'https://www.khaleejtimes.com/uae', 'https://www.khaleejtimes.com/uae/local-plus', 'https://www.khaleejtimes.com/weather', 'https://www.khaleejtimes.com/guide-to-uae', 'https://www.khaleejtimes.com/life-and-living/getting-a-loan', 'https://www.khaleejtimes.com/life-and-living/schools-in-the-uae', 'https://www.khaleejtimes.com/life-and-living/renting-in-the-uae', 'https://www.khaleejtimes.com/ramadan-2024', 'https://www.khaleejtimes.com/world', 'https://www.khaleejtimes.com/asia/india', 'https://www.khaleejtimes.com/asia/pakistan', 'https://www.khaleejtimes.com/world/americas', 'https://www.khaleejtimes.com/world/europe', 'https://www.khaleejtimes.com/world/africa', 'https://www.khaleejtimes.com/opinion', 'https://www.khaleejtimes.com/business', 'https://www.khaleejtimes.com/business/auto', 'https://www.khaleejtime

## Joining sub urls into one single list

In [4]:
website_urls = [item for sublist in list(inside_urls.values()) for item in sublist]
print(len(website_urls))

website_urls = website_urls[299:]
len(website_urls)

337


38

## Creating {url : html content} dictionary

In [5]:
url_html_extracted = get_html(website_urls)
url_html_extracted

100%|██████████| 38/38 [00:47<00:00,  1.26s/it]


({'https://www.khaleejtimes.com/gold-and-forex': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGold and Forex Latest News and Headlines | Khaleej Times\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSign In\n\n\n\n\n\nMy KT\nTrading\nContact Us\nPrivacy Policy\nSign Out\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSun, Dec 31, 2023 | Jamada Al-Aakhirah 18, 1445\n\n\n\n\n\n\n\n\n00:00:00\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nUAE\n\n\n\n\nLocal Plus\nCrime\nLegal\nEducation\nExpo City Dubai\nGovernment\nTransport\nWeather\nEmergencies\nDubai\nAbu Dhabi\nSharjah\nAjman\nRas Al Khaimah\nFujairah\nUmm Al Quwain\n\n\n\n\n\n\nLife and Living\n\n\n\n\nVisa & Immigration in UAE\nBanking in UAE\nSchooling in UAE\nHousing in UAE\nRamadan 2024\n\n\n\n\n\n\nWorld\n\n\n\n\nSaudi Arabia\nBahrain\nOman\nKuwai

## Keyword extraction performed on above dictionary

In [6]:
keywords = ["billion"]
url_extracted_html = kep(website_content = url_html_extracted[0], keywords = keywords, filter_by_amount = 60)


url_extracted_html

100%|██████████| 38/38 [00:00<00:00, 5520.73it/s]


{'https://www.khaleejtimes.com/entertainment': "movies‘Barbie’ joins $1 billion club, breaks another record for female directorsIn just three weeks in theatres, “Barbie” is set to join other 53 movies that have made over $1 billionmoviesentertainmentSunny Deol, Ameesha Patel visit Attari–Wagah border ahead of Gadar 2 releaseThe sequel to the superhit movie Gadar: Ek Prem Katha will be released on August 11 entertainmententertainmentSalman Khan's Tiger 3 to have 'Avengers: Endgame' connection?Bollywood star Shah Rukh Khan reportedly has a cameo in the much-awaited film as wellentertainmentmoviesVideo: Shah Rukh Khan shares daughter Suhana's debut film The Archies teaserThe Bollywood superstar wishes his daughter all the best for the movie to be released on Netflixmoviesmovies'American Fiction' wins top prize at Toronto film festmovies‘Blue Beetle’ unseats ‘Barbie’ atop box office, ending four-week reignBarbie nearly managed to stay No. 1 again with $21.5 million in its fifth weekend aga